In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
import os
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [2]:
%%time
path = '/glade/u/home/noteng/work/research/data/'
file = 'march13-march14.nc'
data = xr.open_dataset(path+file)
data = data.sel(time=slice('2020-03-13T04:00:00.000000000', '2020-03-14T05:00:00.000000000'))
data.close()

CPU times: user 48.7 ms, sys: 15 ms, total: 63.7 ms
Wall time: 247 ms


### equivalent reflectivity factor

In [3]:
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,450:580,256:771] #Based on longitude and latitude of Andoya and Norwegian Sea
equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,250:650,450:850] #Based on longitude and latitude of Andoya and Norwegian Sea
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,330:580,660:780] #### hdm1 and hdm2
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor']
equivalent_reflectivity_factor

<xarray.DataArray 'equivalent_reflectivity_factor' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
    lon      (Yc, Xc) float32 ...
    lat      (Yc, Xc) float32 ...
Attributes:
    standard_name:  equivalent_reflectivity_factor
    long_name:      Radar Reflectivity
    units:          dBZ
    grid_mapping:   projection_lambert

### convert equivalent reflectivity factor to Iris cube

In [4]:
%%time
ERF = equivalent_reflectivity_factor.to_iris()
ERF

CPU times: user 3.94 s, sys: 237 ms, total: 4.18 s
Wall time: 4.37 s


Equivalent Reflectivity Factor (dBZ),time,projection_y_coordinate,projection_x_coordinate
Shape,301,400,400
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
latitude,-,x,x
longitude,-,x,x


In [5]:
%%time
# Determine temporal and spatial sampling of the input data:
#grid_spacing = 1km... but tobac uses meters... 1000m = 1km
#time_spacing = 5 minutes time resolution... tobac uses seconds..... 
#since our time_spacing is 5 min, we get our time spacing in seconds.. if 60 sec = 1 min? then 5 mins = 300s... so time-spacing is 300
dxy,dt=tobac.utils.get_spacings(ERF, grid_spacing=1000, time_spacing=300)  #tobac detect it by default
dxy, dt 

CPU times: user 46 µs, sys: 5 µs, total: 51 µs
Wall time: 55.3 µs


(1000, 300)

# DETECTION FEATURE

In [6]:
equivalent_reflectivity_factor.max(), equivalent_reflectivity_factor.min()

(<xarray.DataArray 'equivalent_reflectivity_factor' ()>
 array(48.166157, dtype=float32),
 <xarray.DataArray 'equivalent_reflectivity_factor' ()>
 array(-32.408, dtype=float32))

In [7]:
# Upon experimenting, i found out that threshold of 10, 15 and 20 dBZ performed best or gave the best result
# threshold of 10 was ranked 1st, then 15, 20. Since 10 performed well, we will use that as  target to other things(segmentation-threshold)
# and tracking v-max

In [8]:
np.arange(-30, 40, 5)

array([-30, -25, -20, -15, -10,  -5,   0,   5,  10,  15,  20,  25,  30,
        35])

In [8]:
%%time
# threshold = np.arange(-15, 35, 5) 
threshold = np.arange(-30, 35, 5)
# threshold = [-30, -25, -20, -15, -10, -5, 0, 5, 10, 15, 20, 25, 30]
parameters_features = {}
parameters_features['target'] = 'maximum'
parameters_features['threshold'] = threshold
parameters_features['n_min_threshold'] = 0 #set to zero or one always; 
parameters_features['n_erosion_threshold'] = 0 #another filtering/smoothing method.
parameters_features['position_threshold'] ='weighted_diff'
parameters_features['sigma_threshold'] = 0.85 #smoothing data
# parameters_features['min_distance'] = 15

# Using 'center' here outputs the feature location as the arithmetic center of the detected feature
Features = tobac.feature_detection_multithreshold(field_in=ERF, dxy=dxy, **parameters_features)

CPU times: user 44.4 s, sys: 62.1 ms, total: 44.5 s
Wall time: 45.6 s


In [9]:
%%time
Features.head()

CPU times: user 104 µs, sys: 2 µs, total: 106 µs
Wall time: 108 µs


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude
0,0,399,222.481123,301.518877,2,-5,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.282481e+06,218518.877030,69.351172,15.468683
1,0,418,284.000000,229.000000,1,-5,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.344000e+06,146000.000000,68.844109,13.564160
2,0,423,310.727163,278.000000,2,-5,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.370727e+06,195000.000000,68.568710,14.702179
3,0,444,39.000000,369.000000,1,0,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.099000e+06,286000.000000,70.939285,17.759075
4,0,453,121.000000,324.705104,2,0,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.181000e+06,241705.104259,70.248601,16.323888


In [10]:
Features.to_csv('../saved-files/threshold-all/Features-all-all.csv', index=False)

In [11]:
Features['threshold_value'].max()

30

# SEGMENTATION

In [12]:
%%time
# Keyword arguments for the segmentation step:
parameters_segmentation={}
parameters_segmentation['target']='maximum'
parameters_segmentation['method']='watershed'
parameters_segmentation['threshold']= -30

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs


In [13]:
%%time
# Perform segmentation and save results to files:
Mask_ERF, Features_ERF = tobac.segmentation_2D(Features,ERF,dxy,**parameters_segmentation)

CPU times: user 2min 6s, sys: 462 ms, total: 2min 7s
Wall time: 2min 11s


In [14]:
type(Mask_ERF)

iris.cube.Cube

In [15]:
iris.save(Mask_ERF, '../saved-files/threshold-all/Mask_ERF_iris-all-all.nc')

In [16]:
%%time
Mask_ERF

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


Segmentation Mask (1),time,projection_y_coordinate,projection_x_coordinate
Shape,301,400,400
Dimension coordinates,,,
time,x,-,-
projection_y_coordinate,-,x,-
projection_x_coordinate,-,-,x
Auxiliary coordinates,,,
latitude,-,x,x
longitude,-,x,x


In [17]:
%%time
# Convert the segmentation data from iris cube to DataArray
segmented_data = xr.DataArray.from_iris(Mask_ERF)
segmented_data

CPU times: user 3.17 ms, sys: 1e+03 ns, total: 3.17 ms
Wall time: 3.17 ms


<xarray.DataArray 'segmentation_mask' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=int32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
    lat      (Yc, Xc) float32 ...
    lon      (Yc, Xc) float32 ...
Attributes:
    long_name:  segmentation_mask

In [18]:
segmented_data.to_netcdf('../saved-files/threshold-all/segmented_ERF-xr-all-all.nc')

# TRAJECTORY LINKING

In [19]:
%%time
# keyword arguments for linking step
parameters_linking={}
parameters_linking['v_max']= -30
parameters_linking['stubs']=1
parameters_linking['order']=1
parameters_linking['extrapolate']=0 
parameters_linking['memory']=0
parameters_linking['adaptive_stop']=0.2
parameters_linking['adaptive_step']=0.95
parameters_linking['subnetwork_size']=36
# parameters_linking['subnetwork_size']=54
parameters_linking['method_linking']= 'predict'
# parameters_linking['time_cell_min'] = 10

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.96 µs


In [20]:
%%time
# Track=tobac.linking_trackpy(Features, ERF, dt=dt, dxy=dxy, **parameters_linking)
Track = tobac.linking_trackpy(Features, ERF, dt=dt, dxy=dxy, **parameters_linking)

Frame 300: 104 trajectories present.
CPU times: user 1min 7s, sys: 90.6 ms, total: 1min 7s
Wall time: 1min 9s


In [21]:
Track.to_csv('../saved-files/threshold-all/Track-all-all.csv', index=False)

In [22]:
# latA = 69.141281 #latitude of COMBLE site
# lonA = 15.684166-1 #longitude of COMBLE site -1

<h1 style="color:red;">TRACKED INFO</h1>

In [23]:
Track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,399,222.481123,301.518877,2,-5,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.282481e+06,218518.877030,69.351172,15.468683,1,0 days 00:00:00
1,0,418,284.000000,229.000000,1,-5,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.344000e+06,146000.000000,68.844109,13.564160,2,0 days 00:00:00
2,0,423,310.727163,278.000000,2,-5,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.370727e+06,195000.000000,68.568710,14.702179,3,0 days 00:00:00
3,0,444,39.000000,369.000000,1,0,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.099000e+06,286000.000000,70.939285,17.759075,4,0 days 00:00:00
4,0,453,121.000000,324.705104,2,0,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.181000e+06,241705.104259,70.248601,16.323888,5,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66394,300,595,159.673195,263.919880,4,30,66395,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.219673e+06,180919.880119,69.951182,14.659732,14895,0 days 01:25:00
66395,300,596,176.000000,380.000000,1,30,66396,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.236000e+06,297000.000000,69.690750,17.566109,14874,0 days 01:30:00
66396,300,597,183.000000,231.000000,1,30,66397,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.243000e+06,148000.000000,69.761421,13.775079,15165,0 days 00:35:00
66397,300,598,230.107923,180.414576,3,30,66398,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.290108e+06,97414.575766,69.357903,12.435729,14990,0 days 01:05:00


In [24]:
%%time
track = Track.sort_values(['cell', 'time_cell'])
track = track.reset_index(drop=True)
track.head()

CPU times: user 20.4 ms, sys: 0 ns, total: 20.4 ms
Wall time: 22.3 ms


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,399,222.481123,301.518877,2,-5,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.282481e+06,218518.87703,69.351172,15.468683,1,0 days 00:00:00
1,0,418,284.000000,229.000000,1,-5,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.344000e+06,146000.00000,68.844109,13.564160,2,0 days 00:00:00
2,0,423,310.727163,278.000000,2,-5,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.370727e+06,195000.00000,68.568710,14.702179,3,0 days 00:00:00
3,0,444,39.000000,369.000000,1,0,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.099000e+06,286000.00000,70.939285,17.759075,4,0 days 00:00:00
4,1,569,39.000000,372.000000,1,5,370,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.099000e+06,289000.00000,70.935585,17.839457,4,0 days 00:05:00


In [25]:
track.to_csv('../saved-files/threshold-all/track-reset-all-all.csv', index=False)

In [26]:
# track = pd.read_csv('saved-files/track-reset.csv')

In [27]:
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,399,222.481123,301.518877,2,-5,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.282481e+06,218518.877030,69.351172,15.468683,1,0 days 00:00:00
1,0,418,284.000000,229.000000,1,-5,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.344000e+06,146000.000000,68.844109,13.564160,2,0 days 00:00:00
2,0,423,310.727163,278.000000,2,-5,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.370727e+06,195000.000000,68.568710,14.702179,3,0 days 00:00:00
3,0,444,39.000000,369.000000,1,0,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.099000e+06,286000.000000,70.939285,17.759075,4,0 days 00:00:00
4,1,569,39.000000,372.000000,1,5,370,2020-03-13 04:05:00,2020-03-13 04:05:00,-2.099000e+06,289000.000000,70.935585,17.839457,4,0 days 00:05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66394,300,511,190.000000,378.523054,2,15,66349,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.250000e+06,295523.054421,69.566104,17.482600,15319,0 days 00:00:00
66395,300,512,192.713674,297.979195,8,15,66350,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.252714e+06,214979.195406,69.624180,15.451297,15320,0 days 00:00:00
66396,300,540,298.000000,225.000000,1,15,66362,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.358000e+06,142000.000000,68.718887,13.446220,15321,0 days 00:00:00
66397,300,563,215.000000,290.000000,1,20,66376,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.275000e+06,207000.000000,69.428795,15.198971,15322,0 days 00:00:00


<h1 style="color:red;  text-align: center;">END OF TRACK</h1>